## Optimizing!



### Introduction to Gurobi Models

Before we get into anything too deep, let’s first take a general look at how optimization works. It's broken down into two main parts: **The objective function** (what we want to solve), and the **constraints** (what limits our input).

The **objective function** tells the program what we want to do, think maximizing profit or minimizing travel time. Let's say in this example, you're buying $x$ hotdogs and $y$ soda cans. You like both equally, and you want as much as possible. So, our **objective function** would look like this:


$$\text{Maximize} \hspace{.2cm} 	x	+	y $$

Now we obviously would want to just make both x and y infinite, but in the real world that obviously isn't allowed, so we need a way to **constrain** the objective function. Let's say in this example a hotdog cost $3 and a soda costs $1.50 and you don't want to spend more than $20. We would write:

\begin{aligned}
3x	+ 1.50y 	&\leq \hspace{.2cm} 20 \\
x	 	 	&\ge \hspace{.2cm}	0 \\
y	 	 	&\ge \hspace{.2cm}	0 \\
\end{aligned}

Putting these together we would typically write the final equation as:

\begin{aligned}
\text{Maximize} \space	x	+	y \\
3x	+ 1.50y 	&\leq \hspace{.2cm} 20 \\
x	 	 	&\ge 	0 \\
y	 	 	&\ge 	0 \\
\end{aligned}

If you're intimidated by the math don't worry, the notation is a formal way to ask how much to buy. Just like machine learning algorithms, while the underlying math can be intimidating, actually applying it is much easier as most of that is abstracted away by the API.

So how do we go about solving this? It's easy, there are three steps:

**1. Instantiate The Solver**

In [14]:
model_1= gp.Model() 

Restricted license - for non-production use only - expires 2025-11-24


**2. Add Our variables**

In [15]:
# Create variables
x = model_1.addVar(vtype=GRB.INTEGER, name="x")
y = model_1.addVar(vtype=GRB.INTEGER, name="y")


**3. Add Our Constraints**

In [16]:
model_1.addConstr(x >= 0, "c0")
model_1.addConstr(y >= 0, "c0")
model_1.addConstr(3*x + 1.5*y <= 20, "c1")


<gurobi.Constr *Awaiting Model Update*>

**4. Add Our Objective Function**

In [17]:
# Set objective
model_1.setObjective(x + y, GRB.MAXIMIZE)


**5. Solve!**

In [18]:
# Optimize model
model_1.optimize()

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[rosetta2] - Darwin 23.6.0 23G80)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 3 rows, 2 columns and 4 nonzeros
Model fingerprint: 0x0d7a692e
Variable types: 0 continuous, 2 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+01, 2e+01]
Found heuristic solution: objective 7.0000000
Presolve removed 3 rows and 2 columns
Presolve time: 0.01s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 2: 13 7 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.300000000000e+01, best bound 1.300000000000e+01, gap 0.0000%


From here we can get all the variables in the model and print their values

In [19]:
#Show Solution
for v in model_1.getVars():
    print('%s %g' % (v.VarName, v.X))
print('Obj: %g' % model_1.ObjVal)


x -0
y 13
Obj: 13


In [20]:
model_2=gp.Model()
# Create variables
x = model_2.addVar(vtype=GRB.INTEGER, name="x")
y = model_2.addVar(vtype=GRB.INTEGER, name="y")

#add constraints
model_2.addConstr(x >= 0, "hotdog_lower_bound")
model_2.addConstr(y >= 0, "soda_lower_bound")
model_2.addConstr(x <= 5, "hotdog_upper_bound")
model_2.addConstr(y <= 5, "soda_upper_bound")
model_2.addConstr(3*x + 1.5*y <= 20, "c1")

model_2.setObjective(2*x + y, GRB.MAXIMIZE)

model_2.optimize()

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[rosetta2] - Darwin 23.6.0 23G80)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 5 rows, 2 columns and 6 nonzeros
Model fingerprint: 0xf66347ca
Variable types: 0 continuous, 2 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [1e+00, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+00, 2e+01]
Found heuristic solution: objective 13.0000000
Presolve removed 5 rows and 2 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 1: 13 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.300000000000e+01, best bound 1.300000000000e+01, gap 0.0000%


In [21]:
#Show Solution
for v in model_2.getVars():
    print('%s %g' % (v.VarName, v.X))
print('Obj: %g' % model_2.ObjVal)


x 4
y 5
Obj: 13


**Now you Try!**

Now that we've introduced the basics, let's try some more problems. They look slightly different from our first example, but hopefully, they'll help you to identify the objective function and constraints. There are some practice questions to help you along as you go, so don't be afraid to make guesses and experiment!

- **Note:** from here on out in the notebook we will note explicitly say our decision variables must be positive (ex. $x>0$) as Gurobi sets this value by default when we declare variables. This can be modified, but in most applications the decision variable is positive.

#### Problem 1

Problem Statement:
Problem Statement: Imagine you are managing disaster relief efforts, and you need to allocate resources between two critical supplies: water bottles ($w$) and emergency food packs ($f$). Your goal is to maximize the number of relief items delivered to an affected area.

- Each water bottle provides essential hydration and is valued at 3 units of relief.
- Each food pack provides vital nutrition and is valued at 5 units of relief.

However, you face the following constraints:

1.	You have a total cargo space that can carry up to 100 units.
2.	Each water bottle takes up 2 units of cargo space.
3.	Each food pack takes up 4 units of cargo space.

In [22]:
problem_1()

 1) What is the Objective Function? 


RadioButtons(options=(('w+f', 1), ('3w+5f', 2), ('2w+4f', 3)), value=1)

Button(description='Check', style=ButtonStyle())

Output()

 2) Which equation represents the cargo space constraint? 


...

Output()

 3) Which equation represents the limit on the number of food packs? 


RadioButtons(options=(('f <= 30', 1), ('w + f <= 50', 2), ('2w + 4f <= 100', 3)), value=1)

Button(description='Check', style=ButtonStyle())

Output()

Great, now try to implement it! Remember you can always look back at the last section and if you get stuck.  We also have the answer key available as you scroll further down (so try not to read ahead)!

**1. Instantiate The Solver**

In [23]:
#Instantiate Here

**2 Add the Variables**

In [24]:
#Add Vars Here

**3 Add the Constraints**

In [25]:
#Add Constraints Here

**4 Add the Objective Function**

In [26]:
#Add Obj. Func. Here

**5 Solve!**

In [27]:
#Solve Here!

##### Problem 1 Answer Key

Remember, this is one possible solution, just because your code doesn't look identical, doesn't mean it's wrong!

In [28]:
#1 Instantiate The Solver
model = gp.Model('DisasterReliefAllocation')

#2 Add the variables for the number of water bottles (w) and food packs (f)
w = model.addVar(vtype=GRB.INTEGER, name='WaterBottles')
f = model.addVar(vtype=GRB.INTEGER, name='FoodPacks')

#3 Add the constraint for the total cargo space
model.addConstr(2 * w + 4 * f <= 100, 'CargoSpace')

#4 Add the objective function: Maximize the total value of relief items
model.setObjective(3 * w + 5 * f, GRB.MAXIMIZE)

#5 Optimize the model
model.optimize()

# Print the optimal solution
if model.status == GRB.OPTIMAL:
    print(f"Optimal number of Water Bottles (w): {int(w.x)}")
    print(f"Optimal number of Food Packs (f): {int(f.x)}")
    print(f"Maximum Relief Value: {model.objVal}")
else:
    print("No optimal solution found.")

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[rosetta2] - Darwin 23.6.0 23G80)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 1 rows, 2 columns and 2 nonzeros
Model fingerprint: 0x451e4b4f
Variable types: 0 continuous, 2 integer (0 binary)
Coefficient statistics:
  Matrix range     [2e+00, 4e+00]
  Objective range  [3e+00, 5e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+02, 1e+02]
Found heuristic solution: objective 150.0000000
Presolve removed 1 rows and 2 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 1: 150 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.500000000000e+02, best bound 1.500000000000e+02, gap 0.0000%
Optimal number of Water Bottles (w): 50
Optimal number of Food Packs (f): 0
Maximum Relief Value: 150.

#### Problem 2

Before diving into the next example, we wanted to take a second to mention a great resource by an expert on the Gurobi team talking about implementing models. It's a little more comprehensive than our current examples, but if you want to see how an expert thinks about this problem, [it's incredibly useful!](https://www.gurobi.com/resources/optimization-modeling-the-art-of-not-making-it-an-art/)

- **Note:** NOTE: The following example is based on examples from [Gurobi’s MOOC for Udemy](https://www.gurobi.com/resources/intro-to-optimization-through-the-lens-of-data-science/)

A non-govermental-organization (NGO) and a manufacturing firm are partnering up to produce supplies to prepare for hurricane season. The NGO predicts that there will be a maximum demand of up to 100 packs of blankets, 70 packs of towels, and 40 packs of sleeping bags. Let's say making these items requires two precision machining steps: weaving and packaging. The table below shows how many minutes are to produce each type of disaster relief item:

|          | Weaving    | Packaging | 
| -------- | ------- | -------|
| Blankets  | 10    |15   |
| Towels | 8    |18  |
| Sleeping Bags    | 12    |17   |

To allow for maintenance and downtime, the company does not want to run its machines beyond a certain limit. The total time available on the machines is 2,500 hours for weaving and 2,000 hours for packaging.

Once items are manufactured, they go to a quality control tester. They are contracted to test exactly 150 items for this upcoming season, no more and no less. Therefore, the company must manufacture exactly 150 items.

Now for the final step, let's consider how many people each pack of items can satisfy:

|          | People Served     | 
| -------- | ------- |
| Blankets  | 6   |
| Towels | 7   |
| Sleeping Bags    | 10  |

In [155]:
#Implement your solution here

##### Problem 2 Answer Key

In [29]:
m = gp.Model("instrument") 
 
# make decision Var
# this is the qunatity of each instrument
x = m.addVars(3, vtype=GRB.INTEGER,name="x") 
m.setObjective(6*x[0] + 7*x[1] + 10*x[2],GRB.MAXIMIZE) 

m.addConstr(10*x[0] + 8*x[1]  + 12*x[2] <= 2000, name='Weaving') 
m.addConstr(15*x[0] + 18*x[1] + 17*x[2] <= 2400, name='Packaging') 
m.addConstr(x.sum() == 150, name='Total_Supplies') 
 
m.addConstr(x[0] <= 100, name='max_100_Blankets_demand') 
m.addConstr(x[1] <= 70, name='max_70_Towels_demand') 
m.addConstr(x[2] <= 40, name='max_40_SleepBags_demand') 
m.optimize()


Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[rosetta2] - Darwin 23.6.0 23G80)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 6 rows, 3 columns and 12 nonzeros
Model fingerprint: 0x00f9ccc8
Variable types: 0 continuous, 3 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [6e+00, 1e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e+01, 2e+03]
Presolve removed 6 rows and 3 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 1: 1083 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.083000000000e+03, best bound 1.083000000000e+03, gap 0.0000%


In [30]:
#Show Solution
a=0
for v in m.getVars():
    print('%s %g' % (v.VarName, v.X))
    a+=v.X
print('Obj: %g' % m.ObjVal)
print(a)


x[0] 87
x[1] 23
x[2] 40
Obj: 1083
150.0


In [31]:
binding = [c for c in m.getConstrs() if abs(c.Slack) < 1e-6]
print(binding)

[<gurobi.Constr Total_Supplies>, <gurobi.Constr max_40_SleepBags_demand>]


In [32]:
for c in m.getConstrs():
    print(c.Slack)

466.0
1.0
0.0
13.0
47.0
0.0


### 2.2 Intro to the ESUPS Optimization Model


Now that you're familiar with the basics, let's start looking at the actual model. We'll start with a simplified version of the model, and we'll build from there.

Consider the scenario where we're expecting to have a disaster and want to pre-position supplies in order to respond as quickly as possible. Perhaps it's hurricane season or a volcano has been showing increasing activity. As we have seen in past problems, we have to set up the variables, constraints, and objective functions. So first let's define our decision variables, i.e. the things we can vary.

Let:

- $\tau_{ij}$ be the time to ship a single item from warehouse $i$ to the disaster

- $y_i$ be the amount of supplies to send from warehouse $i$ to our disaster

- $x_i$ be the starting inventory at each warehouse

- $d$ be the demand for an item


Now that we have described what we can change with our variables, we can figure out how to represent the objective function!
$$
\min_y \sum_i \tau_{i}\cdot y_i

$$

and the constraints are:

$$
\begin{aligned}


\text{s.t.}  & \sum_{i} & y_{i}&=d & & \hspace{.2cm} \text{(total supplies sent must meet demand)}\\

& & y_i       &\leq x_i & \forall i \in I& \hspace{.2cm}\text{(you can't send more than a warehouse has)}\\

 &\text{} & y_{i} &\geq 0 &\forall i \in I& \hspace{.2cm} \text{(you can't send negative supplies)}
\end{aligned}
$$

So now let's solve this model. 

##### Make The model

In [ ]:
#prep distance matrix
df_distance, relevant_warehouses, BucketsNeeded = get_distance_matrix(dataset)
t = df_distance.drivingTime_hrs
n = len(relevant_warehouses)

# Create model
model = gp.Model("simple_Allocation") 

#Add Decision Var
y=model.addVars(n, vtype=GRB.INTEGER, name="Warehouse_Allocation")

#Add constraint to meet demand
model.addConstr(gp.quicksum(y[i] for i in range(n))==BucketsNeeded,name='Meet_Demand')

# Add in warehouse_constraints
for i, supplies in enumerate(relevant_warehouses):
    model.addConstr(y[i] <= supplies[2], name=f"warehouse_endowment_{i}")

# Add objective
objective = gp.quicksum(t[i] * y[i] for i in range(n))
model.setObjective(objective, GRB.MINIMIZE)


In [64]:
model.optimize()

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[rosetta2] - Darwin 23.6.0 23G80)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 33 rows, 32 columns and 48 nonzeros
Model fingerprint: 0xfec0ae5a
Model has 16 quadratic objective terms
Variable types: 0 continuous, 32 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e+00, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+00, 1e+04]
Presolved: 1 rows, 13 columns, 13 nonzeros

Continuing optimization...


Explored 1 nodes (1 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 8 (of 8 available processors)

Solution count 3: 98293 185480 185541 

Optimal solution found (tolerance 1.00e-04)
Best objective 9.829300000000e+04, best bound 9.829300000000e+04, gap 0.0000%


Now Let's Anaylize the results!

In [ ]:
#Show Solution
b=[]
for v, total, dis in zip(model.getVars(),relevant_warehouses,list(df_distance['drivingTime_hrs'])):
    if v.VarName[0:20]== 'Warehouse_Allocation':
        #print('%s %g | total= %g | distance=%g' % (v.VarName, v.X,total[2],dis))
        b.append([v.VarName, v.X,total[2],dis])
b=pd.DataFrame(b)
b.columns=['Var','amount','possible','distance' ]
b.sort_values('distance')

,Var,amount,possible,distance
1,Warehouse_Allocation[1],26.0,26,0.00
4,Warehouse_Allocation[4],9046.0,9046,6.00
11,Warehouse_Allocation[11],3.0,3,7.00
14,Warehouse_Allocation[14],1580.0,1580,8.00
5,Warehouse_Allocation[5],610.0,610,10.00
2,Warehouse_Allocation[2],-0.0,41,11.00
7,Warehouse_Allocation[7],2296.0,6689,11.00
0,Warehouse_Allocation[0],0.0,375,14.00
9,Warehouse_Allocation[9],-0.0,2460,14.25
8,Warehouse_Allocation[8],0.0,150,16.00


Now you may have noticed that this feels like overkill. If we want to position supplies to respond to a known disaster, you might think that we should just put them as close as possible. It's an intuitive solution that can be solved with a simple greedy algorithm. But of course, life is never that simple. 

Now that we've got the initial problem outlined, let's start making it more realistic with two additions:
1.	Instead of preparing for only one disaster, let's prepare for all the disasters that might occur.
2.	Instead of being an omniscient observer, let's say we aren't sure where the next disaster will be




Now, these assumptions might be intimidating at first. How do you rigorously model information we don't know? Isn't this the realm of predictive models?

This reaction is totally normal and indeed we often use predictive modeling when our information is incomplete, however, remember that this problem isn't easily suited to ML techniques due to the low availability of data, small feature space, and large solution space. So how do we approach this with optimization? Well now that we know how supplies will be dispensed for a given warehouse allocation, we can leverage the small set of historical disasters we do have to ask which allocations would have saved/minimized the time taken to respond to all of the disasters. Then our optimization techniques will give us a small number that could feasibly satisfy this (where our constraints meet, see the introduction for more on this) and we can just iterate over them.

So how do we go about it in practice?

### Including All Disasters

Before we begin, let's explicitly define our new problem with the additional requirements outlined in the previous section so we're all on the same page. Our first step is to add in the fact that there are more disasters than just one. We can do that by including a variable to denote which disaster we're talking about

Let:

- $k$ the disaster scenario at hand i.e. a storm, earthquake, or epidemic

- $j$ be the location of the disaster

In scenario $k$ with the disaster located at $j$, the time for a warehouse $i$  to send $y_i$ items is 

 $$\tau_{ij}\cdot y^k_i$$

Repeating for all warehouses gives us

$$  \sum_i \tau_{ij}\cdot y^k_i $$


Repeating for all disasters gives us

$$ \sum_k \sum_i \tau_{ij}\cdot y^k_i $$


This formulation doesn't change any of the solutions we found in the last section, instead, it's leveraging the power of our notation to be able to solve for the optimal allocation for every disaster in one fell swoop! It's been a while since we looked at the original problem in its entirety, so let's take a step back to understand why this is so important.

The question that ESUPS, and subsequently this model, set out to solve is: where should disaster relief supplies be stored in a country to ensure it is as fast and cheap as possible to get them to people in need in the event of a disaster<sup>1</sup>? By comparing the overall travel time for every single disaster that we have data on, we can make determinations on whether storing different quantities of materials in different warehouses would be better or worse given our historical data.


<sup>1: Remember that the model we're looking at is specifically for time, to solve for cost we would replace our time variable with a cost variable  </sup> $^{\tau_{ij}}$ <sup>with a cost variable </sup> $^{c_{ij}}$

Before we get to solving, let's add in one more small factor. Different disasters occur at different rates. A landlocked nation may be less likely to experience a disaster caused by a tropical storm than an earthquake, so shouldn't we weigh the response time to earthquakes more than that of a storm?

### Accounting for Randomness

One of the most difficult tasks that we can encounter in data science problems is randomness or as it's often called "stochastic" elements. This can be seen in all kinds of ways, but in our case study, we're going to look at how we account for not knowing which disaster will hit next or even several years in the future. The way we do this is to create a stochastic model, which may initially seem intimidating, but in discrete events like this, it's super easy.

If you're familiar with expected value this will quickly make sense, but no need to have any prior experience! The idea is that we weigh the outcome of the event (i.e. total travel time in this case) by the probability it occurs. So, if an earthquake is 3 times as likely as a flood, we would rewrite the equation as:

$$.75 \cdot\text{(travel time earthquake)} + .25 \cdot\text{(travel time flood)} $$


#### More On Expected Value

If you haven't seen expected value before or if it's just a been a while and you'd like a refresher, try some of the practice problems below!

When we solve expected value for discrete outcomes, we take the value/outcome/payoff of each possible event and discount it by the probability that it actually occurs. So it follows the form

$$
E[X]=p_1(x_1)+p_2(x_2)+ ... + p_n(x_n)
$$
To simplify this notation we typically write this as a series:

$$
E[X]=\sum_{i=1}^{i=n} p_i \cdot x_i
$$

Try some of the problems below to get a better feel for it!

**Starter Problems**

##### Coin Flip: 

Suppose we play a coin flipping game. Every time we flip the coin and it lands on heads, you get a dollar, and every time it lands on tails, you have to pay a dollar. How much are you expected to make or lose when playing this game?

In [37]:
q_3_1()

 1) What is the Objective Function? 


RadioButtons(options=(('-1', 1), ('0', 2), ('1', 3)), value=1)

Button(description='Check', style=ButtonStyle())

Output()

##### Coin Flip Varient: 

Now, let's change the game a little, say you now get 5 paid dollars when the coin lands on heads and only have to pay 3 dollars when it lands on tails. Would you play? How much are you expected to make or lose when playing this game?

In [38]:
q_3_2()

 1) What is the Objective Function? 


RadioButtons(options=(('0', 1), ('1/2', 2), ('1', 3)), value=1)

Button(description='Check', style=ButtonStyle())

Output()

##### Simple Dice Game:


Suppose you roll a fair six-sided die. If it lands on 6, you win $10; otherwise, you lose $2. What is the expected value of this game? Should you play it?

##### Solution


$$
\begin{aligned}
E[X] &= \left(\frac{1}{6} \times 10\right) + \left(\frac{5}{6} \times (-2)\right)
\\
E[X] &= \left(\frac{10}{6}\right) + \left(\frac{-10}{6}\right)
\\
E[X] &= \frac{10 - 10}{6} = 0
\end{aligned}
$$
**Answer**: The expected value is $\$0$. This means, on average, you neither gain nor lose money from playing this game.


##### Two-Coin Game:


You flip two fair coins. If both coins land on heads, you win $8. If one coin lands on heads and the other on tails, you win $4. If both coins land on tails, you lose $5. What is the expected value of playing this game?


##### Solution:


- Probability of two heads (HH): $\frac{1}{4}$
- Probability of one head and one tail (HT or TH): $\frac{1}{2}$
- Probability of two tails (TT): $\frac{1}{4}$

$$
\begin{aligned}
E[X] &= \left(\frac{1}{4} \times 8\right) + \left(\frac{1}{2} \times 4\right) + \left(\frac{1}{4} \times (-5)\right)
\\
E[X] &= \left(2\right) + \left(2\right) + \left(-1.25\right)
\\
E[X] &= 2 + 2 - 1.25 = 2.75

\end{aligned}
$$
**Answer**: The expected value is $\$2.75$. This means, on average, you gain $\$2.75$ per game.


##### Intermediate Problems

**Custom Die Game:**
Imagine a custom die with faces labeled 1, 2, 3, 4, 5, and 6. Each face has a different probability: \(P(1) = 0.1\), \(P(2) = 0.2\), \(P(3) = 0.15\), \(P(4) = 0.25\), \(P(5) = 0.2\), and \(P(6) = 0.1\). If the outcome of the die is 1, you win $2; for a 2, you win $4; for a 3, you win $6; for a 4, you lose $3; for a 5, you lose $7; and for a 6, you lose $5. What is the expected value of this game?

##### Solution:
$$
\begin{aligned}
E[X] &= (0.1 \times 2) + (0.2 \times 4) + (0.15 \times 6) + (0.25 \times -3) + (0.2 \times -7) + (0.1 \times -5)

\\
E[X] &= (0.2) + (0.8) + (0.9) + (-0.75) + (-1.4) + (-0.5)
\\

E[X] &= 1.9 - 2.65 = -0.75

\end{aligned}
$$
**Answer**:The expected value is $-\$0.75$. On average, you lose $75$ cents per roll.


**Card Drawing Game:**
You draw a card from a standard deck of 52 playing cards. If you draw an Ace, you win $15. If you draw a King, Queen, or Jack, you win $5. For any other card, you lose $3. What is the expected value of this game?

##### Solution:

- Probability of drawing an Ace: $\frac{4}{52} = \frac{1}{13}$
- Probability of drawing a King, Queen, or Jack: $\frac{12}{52} = \frac{3}{13}$
- Probability of drawing any other card: $\frac{36}{52} = \frac{9}{13}$

$$
\begin{aligned}

E[X] &= \left(\frac{1}{13} \times 15\right) + \left(\frac{3}{13} \times 5\right) + \left(\frac{9}{13} \times (-3)\right)
\\


E[X] &= \left(\frac{15}{13}\right) + \left(\frac{15}{13}\right) + \left(\frac{-27}{13}\right)
\\


E[X] &= \frac{15 + 15 - 27}{13} = \frac{3}{13}

\\
E[X] &\approx 0.23

\end{aligned}
$$

**Answer**: The expected value is approximately \$0.23. On average, you gain 23 cents per draw.



##### Advanced Extension Problems 

5. **Roulette Game:**
   You bet $1 on a single number in a game of American Roulette. If the ball lands on your number, you win $35; otherwise, you lose your bet. There are 38 slots on an American Roulette wheel (numbers 1-36, 0, and 00). What is the expected value of this bet?

6. **Insurance Policy:**
   An insurance company sells a one-year term life insurance policy for $500 to a healthy 30-year-old. The policy pays $100,000 in case of death within the year. The probability of a healthy 30-year-old dying within a year is 0.001. What is the expected profit for the insurance company from selling this policy?

7. **Stock Investment Scenario:**
   Suppose you invest in a stock with three possible outcomes over a year: there is a 50% chance it will increase by 10%, a 30% chance it will decrease by 5%, and a 20% chance it will decrease by 15%. What is the expected value of the return on investment after one year?


#### The Model

Now that we've had some background on expected value, let's return to our model! Remeber out goal is to account for the inherent randomness associated with diffrent disaster occuring  

Let $P^k$ be the probability of disaster $k$ and $t^k$ be the total travel time involved in disaster $k$ in the previous sections. Using our definition of expected value, this gives us

$$ \sum_k P^k \cdot t^k$$

Now you might notice that we have already written an equation for the total travel time for disaster $k$ ! Substituting this in we get

$$\sum_k P^k \sum_i \tau_{ij}\cdot y^k_i$$

So, you can see, in our case turning this problem into a stochastic optimization problem that can account for probability only involves one more variable. So, our final task right now is to minimize the time required to get supplies to a disaster given how likely each disaster is to occur:


$$
\min_y \sum_k P^k \sum_i \tau_{ij}\cdot y^k_i

$$

And we can see the constraints are almost unchanged. The only added part is that all probabilities must sum to one, which is always the case:
$$
\begin{aligned}


\text{s.t.}  & \sum_{i} & y^k_{i}&=d^k & & \hspace{.2cm} \text{(total supplies sent must meet demand)}\\

& & y^k_i       &\leq x_i & \forall i \in I& \hspace{.2cm}\text{(you can't send more than a warehouse has)}\\

 &\text{} & y^k_{i} &\geq 0 &\forall i \in I& \hspace{.2cm} \text{(you can't send negative supplies)}\\

& \sum_k & P^k &=1 && \hspace{.2cm} \text{(All probabilities must sum to 1)}\\


\end{aligned}
$$

How do we calculate the probability though? Well, we have good long-term data on what disasters have affected which countries. For now, we can go through that data and calculate the probability of disaster k by counting how many times it has occurred and dividing by the number of total disasters i.e.

$$ P^k = \frac{k}{\|K\|}$$

You may have noticed that every single disaster will have the same probability, or in other words, this will produce a uniform distribution. As a result, $P^k$ is a constant and can effectively be removed, so our new equation will give the exact same results as our old one. We'll talk about ways to potentially address this later on in the notebooks as an extension, but for the moment, the purpose of adding this is to show how easily the equation can be modified to be stochastic and to get users used to the idea.

So now that we have the equation, we can let the model go ahead and let Gurobi solve it for us

We will start using ```with``` statements to accomidate the model's growing size and need for our License

In [ ]:
demand, probs = get_probs(dataset)

n = len(relevant_warehouses)
m = len(demand)
a = []

# Create an array of driving times based on the df_distance DataFrame
t = [int(row['drivingTime_hrs']) for i, row in df_distance.iterrows()]
c=pd.DataFrame(t)
print(t)

# Amount to take per Warehouse
y = model.addVars(m, n, vtype=GRB.INTEGER, name="Warehouse_Allocation")

# Add constraints to meet demand for each disaster scenario (k)
for k in range(m):
    # Demand constraints
    model.addConstr(gp.quicksum(y[k, i] for i in range(n)) == demand[k], name=f"Meet_Demand_K:{k}")
    
    # Warehouse constraints
    for i, supplies in enumerate(relevant_warehouses):
        model.addConstr(y[k, i] <= supplies[2], name=f"warehouse_endowment_K:{k}_I:{i}")

# Objective function to minimize the weighted driving time using T as a parameter
objective = gp.quicksum(
    probs[k] * gp.quicksum(t[i] * y[k, i] for i in range(n))
    for k in range(m)
)

# Optimize model
model.setObjective(objective, GRB.MINIMIZE)
model.optimize()

# Store results in the list 'a'
for v in model.getVars():
    a.append([v.VarName, v.X])

[14, 0, 11, 26, 6, 10, 19, 11, 16, 14, 17, 7, 16, 23, 8, 22]
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[rosetta2] - Darwin 23.6.0 23G80)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 1088 rows, 1024 columns and 2048 nonzeros
Model fingerprint: 0xad7c1256
Variable types: 0 continuous, 1024 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [9e-02, 4e-01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+00, 4e+04]
Presolve removed 1087 rows and 1011 columns
Presolve time: 0.01s
Presolved: 1 rows, 13 columns, 13 nonzeros
Variable types: 0 continuous, 13 integer (0 binary)
Found heuristic solution: objective 330077.56250
Found heuristic solution: objective 329939.28125

Root relaxation: objective 3.270432e+05, 1 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Dept

In [50]:
pd.DataFrame(a).iloc[:15]

,0,1
0,"Warehouse_Allocation[0,0]",375.0
1,"Warehouse_Allocation[0,1]",26.0
2,"Warehouse_Allocation[0,2]",41.0
3,"Warehouse_Allocation[0,3]",2322.0
4,"Warehouse_Allocation[0,4]",9046.0
5,"Warehouse_Allocation[0,5]",610.0
6,"Warehouse_Allocation[0,6]",5201.0
7,"Warehouse_Allocation[0,7]",6689.0
8,"Warehouse_Allocation[0,8]",150.0
9,"Warehouse_Allocation[0,9]",2460.0


40811

### 2.5 Data Science Extension

We've seen how the equation uses the uniform distribution to solve the problem, but what if we knew something it didn't? What if knowing that climate change is increasingly energizing large storms, we decide the past hurricane impacts aren't representative of what's to come? In this section we want to prompt you to come up with predictive elements to improve our models. Feel free to use some of the ideas below or go in an entirely new direction!

In this section, we encourage you to think creatively about enhancing predictive models for climate-related disasters. Consider how to incorporate novel data sources, feature engineering techniques, and model architectures to improve predictions. Below are some suggested approaches, but feel free to explore entirely new directions!

Case Study Focus: Coastal Eastern African Nations

Using the disaster impact data for coastal Eastern African nations, can you develop a model to predict how these impacts might escalate for Madagascar in the coming years? Consider not only the historical data but also factors such as changes in sea surface temperatures, shifting storm tracks, population growth along vulnerable coastlines, and evolving infrastructure resilience. Further, can you integrate this predictive model into an optimization framework to better allocate resources for disaster preparedness and response?

Potential Approaches to Explore:
1. Comparing Time Series Models: Traditional statistical time series models like ARIMAX (AutoRegressive Integrated Moving Average with Explanatory Variables) are commonly used to predict future values based on past data. How do these models compare with more advanced Recurrent Neural Network (RNN)-based approaches like Long Short-Term Memory (LSTM) networks or Gated Recurrent Units (GRUs) in capturing long-term dependencies, especially under non-stationary conditions induced by climate change?

2.	Incorporating Geospatial Data: Geospatial features, such as latitude, longitude, elevation, and proximity to bodies of water, can play a crucial role in predicting the impact of tropical storms. Can we encode geospatial information using techniques like convolutional neural networks (CNNs) for spatial feature extraction, or leverage more specialized models such as Geographical Weighted Regression (GWR) or Graph Neural Networks (GNNs) to account for spatial dependencies?

3.	Incorporating Climate Change Projections: Beyond just historical data, consider how future climate projections can be integrated into the model. Can we use downscaled climate model outputs or ensemble approaches to account for different climate scenarios? How would these scenarios affect the frequency and intensity of tropical storms affecting coastal Eastern African nations?

4.	Feature Engineering with Climate Indicators: Introduce climate change indicators as predictive features. For example, how do trends in sea surface temperatures (SSTs), El Niño-Southern Oscillation (ENSO) phases, or the Atlantic Multi-decadal Oscillation (AMO) correlate with storm intensification? Would incorporating these indicators as additional time series variables enhance predictive accuracy?
	
5.	Hybrid and Ensemble Models: Can we leverage hybrid models that combine both statistical and deep learning approaches, or use ensemble methods that aggregate predictions from multiple models? For example, combining ARIMAX for short-term predictions with LSTM for capturing long-term trends may provide a more comprehensive forecasting tool.

6.	Optimization Integration: Once a reliable predictive model is established, how can it be integrated into an optimization framework for resource allocation? For example, can we build an optimization model that minimizes both the cost of disaster preparedness and the potential loss from future storm impacts?

7.	Model Explainability and Decision-Making: How can we ensure that the model is interpretable for decision-makers? Consider the use of techniques like SHAP (SHapley Additive exPlanations) values or LIME (Local Interpretable Model-agnostic Explanations) to explain which factors contribute most to the model’s predictions, helping policymakers make informed decisions.

By combining predictive analytics with optimization, we can not only forecast future disaster impacts but also develop actionable strategies for minimizing those impacts. The goal is to make our models both more accurate and more useful in real-world applications, driving better outcomes for communities at risk.


### Processing the Results

Great, now that we can solve for the time it takes to address every single disaster, we can finally answer the original question posed: how do we best allocate supplies to all the warehouses? The key here is to create a second optimization problem. Remember how in our constraints we included that you can't send more than the warehouse has? 

$$
y^k_i       \leq x_i  \forall i \in I \hspace{.2cm}\text{(you can't send more than a warehouse has)}\\
$$

So far, we've just been using the actual allocation we have at each warehouse for $x_i$. But what if those changes? Suddenly we would have an entirely new solution. So, if we say that the output of Gurobi (i.e. the allocations $y^k_i \in Y$) is some function based on our starting amount warehouses ($X$ where $x_i\in X$), then we can say

$$Y=f(X)$$ 

And when we frame it this way, it becomes much more simple to solve! All we need to do is minimize the travel times $Y$. In other words, our problem becomes 


$$ \min_{X} f(X) $$
$$\begin{aligned}
\text{s.t.}  & \sum_{i} & x_i&=\chi & & \hspace{.2cm} \text{(we allocate all supplies and no more)}\\

 && x_{i} &\geq 0 &\forall i \in I& \hspace{.2cm} \text{(you can't allocate negative supplies)}\\

\end{aligned}$$

Where $\chi$ is the total amount of supplies we have in the country



While this may initially look intimidating, it is one of the easiest changes to make to our current code. All we are doing is making a variable instead of a constraint and then constraining it. Let’s substitute back in our equation from the last section with the new constraints to see this firsthand. To denote a new decision variable (i.e. a variable that can be changed), all we need to do is add it under the minimization sign. This means minimizing with respect to $X$ and $Y$

$$
\min_{X,Y} \sum_k P^k \sum_i \tau_{ij}\cdot y^k_i
$$

Then all we need to do is update the constraints. I've included the line to make it easier to see what's new as our list grows. It has no mathematical significance. 
So how do we Implement this in Gurobi?

$$
\begin{aligned}


\text{s.t.}  & \sum_{i} & y^k_{i}&=d^k & & \hspace{.2cm} \text{(total supplies sent must meet demand)}\\

& & y^k_i       &\leq x_i & \forall i \in I& \hspace{.2cm}\text{(you can't send more than a warehouse has)}\\

 &\text{} & y^k_{i} &\geq 0 &\forall i \in I& \hspace{.2cm} \text{(you can't send negative supplies)}\\

& \sum_k & P^k &=1 && \hspace{.2cm} \text{(All probabilities must sum to 1)}\\

\hline \\

  & \sum_{i} & x_i&=\chi & & \hspace{.2cm} \text{(we allocate all supplies and no more)}\\

 && x_{i} &\geq 0 &\forall i \in I& \hspace{.2cm} \text{(you can't allocate negative supplies)}\\


\end{aligned}
$$



So how do we Implment this in Gurobi?

In [70]:
model = gp.Model("full_allocation")

n = len(relevant_warehouses)
m = len(demand)

# Create an array of driving times based on the df_distance DataFrame
t = df_distance.drivingTime_hrs

# Amount to take per Warehouse
y = model.addVars(m, n, vtype=GRB.INTEGER, name="Single_Warehouse_Allocation")

# National Allocation
X = model.addVars(n, vtype=GRB.INTEGER, name="National_Allocation")

# Total national endowment constraint
model.addConstr(gp.quicksum(X[i] for i in range(n)) == 40811, name="Total_National_Endowment")

for k in range(m):
    model.addConstr(y[k, i] <= X[i])

# Demand and warehouse constraints for each scenario
for k in range(m):
    model.addConstr(gp.quicksum(y[k, i] for i in range(n)) == demand[k], name=f"Meet_Demand_K:{k}")
    for i, supplies in enumerate(relevant_warehouses):
        model.addConstr(y[k, i] <= supplies[2], name=f"warehouse_endowment_K:{k}_I:{i}")

# Objective function to minimize the weighted driving time using T as a parameter
objective = gp.quicksum(
    probs[k] * gp.quicksum(t[i] * y[k, i] for i in range(n))
    for k in range(m)
)

# Optimize model
model.setObjective(objective, GRB.MINIMIZE)
model.optimize()

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[rosetta2] - Darwin 23.6.0 23G80)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 1153 rows, 1040 columns and 2192 nonzeros
Model fingerprint: 0x0e8cfb4c
Variable types: 0 continuous, 1040 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [9e-02, 4e-01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+00, 4e+04]
Presolve removed 1152 rows and 1026 columns
Presolve time: 0.01s
Presolved: 1 rows, 14 columns, 14 nonzeros
Variable types: 0 continuous, 14 integer (0 binary)
Found heuristic solution: objective 330422.47266

Root relaxation: objective 3.274030e+05, 1 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    327403.00000 327403.000  

### A Few Final Tidbits

Now that we have our overall framework, we can extend it fairly easily to better model real-life scenarios. Let's look at two final pieces of the puzzle that ESUPS considers in their model:

**Cost**

Along with how long it takes to get items to a disaster relief site, it's also important to consider the cost to accomplish it. It might be a few hours faster to charter a jet to deliver blankets in the aftermath of a disaster, however, if it is 100x more expensive than by truck, that may constrain the organization from buying more blankets, chartering more trucks, or making it difficult to resupply for future disasters. So just as we solve for ways to minimize time, it can be important for firms with limited resources to make sure their money is being used to do the most good it can.

So how do we do this? It's fairly simple. Our time matrix, which we've been using to show how close or far buildings are from the disaster relief site, is just a set of predefined weights/discounts. So, if we change the numbers to reflect the cost of transit, then suddenly we're solving a cost-minimization problem! In fact, the substitution is so one-to-one, that besides switching $\tau_{ij}$ for $c_{ij}$, we don't have to change the equation.

**Travel Mode**

The second additional facet considered in our real-life model that we haven't encountered yet is transportation mode. We alluded to it a little in the cost section, but often there is the option to fly or ship goods into a region, which can be especially useful when far away or the roads are clogged or otherwise unusable (such is often the case after a disaster).

So how do we implement this? Well let's take a look back at $y_i^k$, our variable which says how many goods to send from warehouse $i$ to disaster $k$. All we want to do is reflect and updated description: how many goods to send from warehouse $i$ to disaster $k$ via mode $r$. This can easily be represented as $y_{ir}^k$, let's explain what's happened. Before is $y$ was an array of length $K$ with each index holding sub array of length $I$ (which we could also write as size $K \times I$), now each index in our subarrays also have an array of length $3$ to represent how much is sent via truck, plane, or boat. So our final array is of dimensions $K \times I \times R $. This may seem intimidating at first, but remember, adding a dimension just means adding one more nested for loop!

Let's look at how we would implement this. Remember, from a math point of view, all we've done is say $y_i^k$ can be broken down into $3$ modes instead of 1. So, it's rewritten as


$$
\min_{X,Y} \sum_k P^k \sum_i \sum_r \tau_{irj}\cdot y^k_{ir}
$$


Then all we need to do is update the constraints. I've included the line to make it easier to see what's new as our list grows. It has no mathematical significance. 
$$
\begin{aligned}


\text{s.t.}  & \sum_{i}\sum_{r} & y^k_{ir}&=d^k & & \hspace{.2cm} \text{(total supplies sent must meet demand)}\\

& \sum_{r}  & y^k_{ir}       &\leq x_i & \forall i \in I& \hspace{.2cm}\text{(you can't send more than a warehouse has)}\\

 &\text{} & y^k_{ir} &\geq 0 &\forall r\in R, i \in I& \hspace{.2cm} \text{(you can't send negative supplies)}\\

& \sum_k & P^k &=1 && \hspace{.2cm} \text{(All probabilities must sum to 1)}\\


  & \sum_{i} & x_i&=\chi & & \hspace{.2cm} \text{(we allocate all supplies and no more)}\\

 && x_{i} &\geq 0 &\forall i \in I& \hspace{.2cm} \text{(you can't allocate negative supplies)}\\


\end{aligned}
$$

We've added a few more sums here, but remember, in math, a sum is just a for loop. $\sum_r$ is the equivalent to `for r in R:`. So how would we implement it in the format we've been using so far? This is going to be left as an open-ended exercise to the reader! If you get stuck, you can reference the production solver we'll be exploring below, which includes the mode of travel but is set up in a different approach than we've been using so far!

#### Open-Ended Implementation

In [168]:
#Write your code here

### Interpreting the Solution 

So let's look at how much slower our real life allocations are in comparison to the optimal.

### Enhancing System Performance with the Balance Metric

In humanitarian logistics, the efficiency of inventory allocation directly impacts the ability to respond swiftly and cost-effectively to disasters. The **Balance Metric** $(D)$ is a critical tool developed to evaluate the alignment of current inventory distribution with an optimal allocation. This metric is particularly valuable in contexts where multiple organizations independently manage inventory across various depots, without a centralized coordination mechanism.

#### Definition and Calculation of the Balance Metric

The balance metric $(D)$ is defined as the ratio between the actual objective value (either cost or time) of the current inventory allocation $V(X)$ and the optimal objective value $V(A')$ given the same overall capacity:

$$ D = \frac{V(A)}{V(A')}  $$

Here:
-  $V(A)$: Represents the current system-wide cost or time to meet demand based on the existing inventory allocation \(X\).
-  $V(A')$: Represents the minimized cost or time if the inventory were optimally distributed across all depots.

#### Interpretation of the Balance Metric

1. **Optimal Inventory Allocation**:
   The optimal value of $D$ is 1. This occurs when the current allocation perfectly aligns with the optimal allocation, meaning no further reallocation could reduce costs or response times.

2. **Identifying Imbalances**:
   When $D > 1$, the system is considered "out-of-balance." A value of 1.2, for example, implies that the current allocation incurs 20% higher costs or longer response times compared to an optimal arrangement. This indicates a potential for improvement by reallocating resources more effectively.

3. **Guiding System Improvements**:
   The balance metric is not only an indicator of inefficiency but also a guide for decision-making. By identifying locations or items with the highest imbalance, decision-makers can prioritize inventory reallocations that would yield the most significant improvements in terms of cost savings or faster response times.

#### Practical Applications in Humanitarian Logistics

The balance metric offers several practical applications for optimizing humanitarian response efforts:

- **Strategic Reallocation of Resources**:
  Organizations can use the balance metric to identify under-stocked or over-stocked depots and adjust inventory levels accordingly. This strategic reallocation can significantly enhance response times or reduce costs, especially in multi-organizational contexts where coordination is limited.

- **Sensitivity to Network Changes**:
  The balance metric is responsive to changes in the logistics network. For example, if a new depot is added in a high-risk area and remains under-stocked, the balance metric will reflect this imbalance, prompting an assessment of whether inventory should be redistributed to better leverage the new depot.

- **Decision-Making in Real-Time Operations**:
  By continuously monitoring the balance metric as part of a real-time dashboard, operational managers can be alerted to changes that may impact overall system performance. This enables them to make data-driven decisions quickly, improving the overall resilience and responsiveness of the humanitarian supply chain.

#### Limitations and Considerations

While the balance metric provides valuable insights into inventory allocation efficiency, it is important to consider its limitations:

- **Impact of Extreme Events**:
  The balance metric can be influenced by extreme scenarios, such as very large-scale disasters that significantly impact the calculated demand. As a result, it should be interpreted alongside other metrics, such as the fraction of demand served ($g$) or the weighted fraction of disasters completely served ($d$), to provide a more comprehensive picture of system performance.

- **Dependence on Data Quality and Model Assumptions**:
  The accuracy of the balance metric depends on the quality of input data and the assumptions made in the model. Ensuring robust and accurate data collection processes and regularly updating model parameters to reflect real-world conditions are essential for maintaining the reliability of the metric.

#### Conclusion

The balance metric $D$ offers a powerful tool for evaluating and improving the efficiency of inventory allocation in humanitarian logistics. By identifying imbalances and guiding strategic reallocation decisions, this metric can help organizations optimize their response efforts, ensuring that resources are used most effectively to meet the needs of affected populations during disasters.

<a id='End'></a>